In [163]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import datetime as dt
pd.set_option('max_columns', 50)
mpl.rcParams['lines.linewidth'] = 2

%matplotlib inline

In [2]:
df_bat = pd.read_csv('sh_battles.csv')
df_ses = pd.read_csv('sh_session.csv')
df_user = pd.read_csv('sh_user.csv')

C:\Program Files\Anaconda2\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Program Files\Anaconda2\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (42) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
print df_bat.shape
print df_ses.shape
print df_user.shape

(397339, 12)
(148724, 40)
(17988, 54)


df_ses =sessio <br />
df_bat = battle<br />
df_user =user

In [4]:
df_ses = df_ses.drop('Unnamed: 0',1)
df_bat = df_bat.drop('Unnamed: 0',1)
df_user = df_user.drop('Unnamed: 0',1)

#dropping misrepresented users
to_drop = [1.0, 2.0]
df_ses = df_ses[~df_ses['user_id'].isin(to_drop)]

df_user = df_user[~df_user['user_id'].isin(to_drop)]

In [5]:
df_bat['datetime']=pd.to_datetime(df_bat['datetime'])
df_ses['datetime']=pd.to_datetime(df_ses['datetime'])

In [10]:
df_ses = pd.merge(df_ses,df_user[['register_source_android','last_global_device_id','register_os','register_device','register_version','register_ip_region','register_ip_city','user_id','revenues_1d','revenues_2d','funnel_5min','funnel_1hour','funnel_1d']],on='user_id')

In [144]:
df_ses=df_ses.drop(['register_device','last_global_device_id','register_source_android','funnel_1d_y','revenues_2w','revenues_1d','revenues_2d'],axis=1)

In [ ]:
# column 13 -client_mobile_os has 132 na ---**drop na**
# column 14 - has 132 na --**drop na**
# column 15 -has 132 na ---**drop na**
# column 16 -has 132 na ---**drop na**
# column 17 -has 6916 na ---- **impute with -99999**
# column 24,25,26,27,28,29,30,31,32,33,34,35,36,37,38 -has 673 na ----- **Drop NA's**
# column 39 -has 77467 nas ---DROP
# column 40 -has 8906 na -- DROP
# column 41 -has 463 na -- DROP  ---duplicate of client_mobile_device
# column 42 -has 463 na -- DROP  ---duplicate of client_mobile_device
# column 46 -has 115678 nas ----DROP
# column 47 -has 115678 nas ----DROP
# column 48 -has 4969 nas ------ **Replace na with mean value**
# column 49 -has 3476 nas ------ **Replace na with mean value**
# column 50 -has 2972 nas ------ **Replace na with mean value**
# column 51 -has 115678 nas -----DROP
# column 52 -has 2972 naS---- DROP --duplicate funnel_1d

In [153]:
df_ses.fillna(value=-99999999999,axis=1,inplace=True)

In [156]:
def cohort_period(df_ses):
    """
    Creates a `CohortPeriod` column, which is the Nth period based on the user's first purchase.
    
    Example
    -------
    Say you want to get the 3rd day for every user:
        df_ses.sort(['user_id', 'OrderTime', inplace=True)
        df_ses = df_ses.groupby('user_id').apply(cohort_period)
        df_ses[df_ses.CohortPeriod == 3]
    """
    df_ses['Cohort_period'] = np.arange(len(df_ses['datetime'])) + 1
    return df_ses

In [158]:
df_ses_new = df_ses.groupby(df_ses['user_id']).apply(cohort_period)

In [166]:
df_ses_new['year'] = df_ses_new['datetime'].dt.year
df_ses_new['month'] = df_ses_new['datetime'].dt.month
df_ses_new['day'] = df_ses_new['datetime'].dt.day

In [168]:
df_ses_new = df_ses_new.drop('datetime',1)

In [172]:
df_save = df_ses_new.to_csv('retention.csv',index=None)

In [171]:
df_ses_new =pd.get_dummies(df_ses_new['platform'])

<class 'pandas.core.frame.DataFrame'>
Int64Index: 124899 entries, 132 to 125030
Data columns (total 49 columns):
user_id                         124899 non-null float64
session_id                      124899 non-null float64
platform                        124899 non-null object
version                         124899 non-null int64
sample_ratio                    124899 non-null int64
ip_ip                           124899 non-null object
ip_country                      124899 non-null object
ip_region                       124899 non-null object
ip_city                         124899 non-null object
ip_lat                          124899 non-null float64
ip_lon                          124899 non-null float64
ip_timezone                     124899 non-null object
client_mobile_os                124899 non-null object
client_mobile_device            124899 non-null object
client_mobile_language          124899 non-null object
client_mobile_uid               124899 non-null object
clien

In [169]:
df_ses_new.describe(include='all')

,user_id,session_id,platform,version,sample_ratio,ip_ip,ip_country,ip_region,ip_city,ip_lat,ip_lon,ip_timezone,client_mobile_os,client_mobile_device,client_mobile_language,client_mobile_uid,client_mobile_device_aid,game_basic_level,game_basic_cash,game_resources_gold,game_resources_food,game_resources_xp,game_resources_stamina,game_resources_de,game_resources_ev,game_resources_fe,game_resources_friend_points,game_resources_le,game_resources_ne,game_resources_rde,game_resources_re,game_resources_rfe,game_resources_rle,game_resources_rne,game_resources_rwe,game_resources_ssre,game_resources_sre,game_resources_we,register_os,register_version,register_ip_region,register_ip_city,funnel_5min,funnel_1hour,funnel_1d_x,Cohort_period,year,month,day
count,1.248990e+05,1.248990e+05,124899,1.248990e+05,124899.0,124899,124899,124899,124899,124899.000000,124899.000000,124899,124899,124899,124899,124899,124899,124899.000000,124899.000000,1.248990e+05,1.248990e+05,1.248990e+05,124899.000000,1.248990e+05,1.248990e+05,1.248990e+05,1.248990e+05,1.248990e+05,1.248990e+05,1.248990e+05,1.248990e+05,1.248990e+05,1.248990e+05,1.248990e+05,1.248990e+05,1.248990e+05,1.248990e+05,1.248990e+05,124899,1.248990e+05,124899,124899,1.248990e+05,1.248990e+05,1.248990e+05,124899.000000,124899.000000,124899.000000,124899.000000
unique,NaN,NaN,2,NaN,NaN,24797,155,782,1763,NaN,NaN,29,28,1679,57,12796,7488,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28,NaN,724,1535,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,NaN,android,NaN,NaN,221.189.28.116,JP,Tokyo,Tokyo,NaN,NaN,+09:00,4.4.2,None,ja,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.4.2,NaN,Tokyo,Tokyo,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,NaN,124780,NaN,NaN,1381,87839,46046,28210,NaN,NaN,87983,41190,29667,76702,29667,29667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38268,NaN,44875,31164,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,3.069190e+18,2.192984e+15,NaN,8.021268e+11,1.0,NaN,NaN,NaN,NaN,32.344651,112.028148,NaN,NaN,NaN,NaN,NaN,NaN,10.740598,73.123083,1.952250e+05,6.582192e+05,1.497716e+05,17.725706,-4.980024e+08,-4.980023e+08,-4.980024e+08,-4.980023e+08,-4.980024e+08,-4.980024e+08,-4.980024e+08,-4.980024e+08,-4.980024e+08,-4.980024e+08,-4.980024e+08,-4.980024e+08,-4.980024e+08,-4.980024e+08,-4.980024e+08,NaN,1.452437e+09,NaN,NaN,-3.957594e+09,-2.763028e+09,-2.359502e+09,67.514392,2014.996813,4.712528,14.405736
std,7.144235e+16,8.166555e+16,NaN,2.829570e+14,0.0,NaN,NaN,NaN,NaN,8.630748,52.632081,NaN,NaN,NaN,NaN,NaN,NaN,9.464509,341.362816,1.306661e+07,2.630514e+06,1.008073e+07,15.221548,7.039363e+09,7.039363e+09,7.039363e+09,7.039363e+09,7.039363e+09,7.039363e+09,7.039363e+09,7.039363e+09,7.039363e+09,7.039363e+09,7.039363e+09,7.039363e+09,7.039363e+09,7.039363e+09,7.039363e+09,NaN,2.753808e+08,NaN,NaN,1.949616e+10,1.639119e+10,1.517845e+10,163.594635,0.056360,0.804882,7.088446
min,1.234500e+04,4.210000e+02,NaN,0.000000e+00,1.0,NaN,NaN,NaN,NaN,-37.850000,-157.858000,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,0.000000,-6.193795e+08,0.000000e+00,0.000000e+00,0.000000,-1.000000e+11,-1.000000e+11,-1.000000e+11,-1.000000e+11,-1.000000e+11,-1.000000e+11,-1.000000e+11,-1.000000e+11,-1.000000e+11,-1.000000e+11,-1.000000e+11,-1.000000e+11,-1.000000e+11,-1.000000e+11,-1.000000e+11,NaN,0.000000e+00,NaN,NaN,-1.000000e+11,-1.000000e+11,-1.000000e+11,1.000000,2014.000000,1.000000,1.000000
25%,3.068037e+18,2.510968e+07,NaN,1.504072e+09,1.0,NaN,NaN,NaN,NaN,33.340600,114.158000,NaN,NaN,NaN,NaN,NaN,NaN,3.000000,12.000000,6.050000e+03,4.200000e+02,1.620000e+03,9.000000,0.000000e+00,1.000000e+01,0.000000e+00,9.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,NaN,1.504072e+09,NaN,NaN,2.080000e+03,5.002000e+03,5.004000e+03,3.000000,2015.000000,4.000000,9.000000
50%,3.073220e+18,4.986946e+07,NaN,1.505082e+09,1.0,NaN,NaN,NaN,NaN,35.66

In [154]:
%timeit 
df_ses.iloc[:,[45]].isnull().values.sum()

0

In [33]:
df_ses = pd.merge(df_ses,df_user[['user_id','revenues_2w','funnel_1d']],on='user_id')

In [147]:
df_ses.head()

,user_id,session_id,datetime,platform,version,sample_ratio,ip_ip,ip_country,ip_region,ip_city,ip_lat,ip_lon,ip_timezone,client_mobile_os,client_mobile_device,client_mobile_language,client_mobile_uid,client_mobile_device_aid,game_basic_level,game_basic_cash,game_resources_gold,game_resources_food,game_resources_xp,game_resources_stamina,game_resources_de,game_resources_ev,game_resources_fe,game_resources_friend_points,game_resources_le,game_resources_ne,game_resources_rde,game_resources_re,game_resources_rfe,game_resources_rle,game_resources_rne,game_resources_rwe,game_resources_ssre,game_resources_sre,game_resources_we,register_os,register_version,register_ip_region,register_ip_city,funnel_5min,funnel_1hour,funnel_1d_x
132,3.064674e+18,43528293.0,2015-03-25,android,0,1,118.200.48.12,SG,Singapore,Singapore,1.28967,103.85,+08:00,None,None,None,None,None,2,33,6050.0,420,340,11,0.0,10.0,0.0,9.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,None,0,Singapore,Singapore,5002.0,5002.0,5002.0
133,3.064674e+18,43528293.0,2015-03-25,android,0,1,118.200.48.12,SG,Singapore,Singapore,1.28967,103.85,+08:00,None,None,None,None,None,2,33,6050.0,420,340,10,0.0,10.0,0.0,9.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,None,0,Singapore,Singapore,5002.0,5002.0,5002.0
134,3.064674e+18,43528293.0,2015-03-25,android,0,1,118.200.48.12,SG,Singapore,Singapore,1.28967,103.85,+08:00,None,None,None,None,None,3,62,6281.0,691,1252,12,0.0,10.0,0.0,3.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,None,0,Singapore,Singapore,5002.0,5002.0,5002.0
135,3.064674e+18,58446297.0,2015-03-26,android,0,1,118.200.48.12,SG,Singapore,Singapore,1.28967,103.85,+08:00,None,None,None,None,None,3,62,6281.0,691,1252,12,0.0,10.0,0.0,3.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,None,0,Singapore,Singapore,5002.0,5002.0,5002.0
136,3.064674e+18,41779711.0,2015-03-26,android,0,1,118.200.48.12,SG,Singapore,Singapore,1.28967,103.85,+08:00,None,None,None,None,None,3,62,6281.0,691,1252,12,0.0,10.0,0.0,3.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,None,0,Singapore,Singapore,5002.0,5002.0,5002.0


In [45]:
len((df_bat['user_id']))

397339

In [58]:
len(set(df_ses['session_id']))

68322

In [15]:
df_new = pd.merge(df_ses,df_user,on = 'user_id')

In [57]:
len(set(df_ses['client_mobile_uid'])),len(set(df_ses['user_id']))

(12812, 17916)

In [37]:
len(set(df_user['last_session_id']))

17984

In [35]:
df_user[df_user['user_id']==3.0732145037735363e+18]

,user_id,last_session_id,date_register,date_register_android,date_last_logged,platform_last_logged,date_last_logged_android,register_platform,register_ip_ip,register_ip_country,register_ip_lat,register_ip_lon,register_ip_timezone,game_current_level,game_current_cash,game_current_gold,game_current_food,game_current_xp,game_current_stamina,revenues_dollars_net,revenues_dollars_gross,revenues_num_transactions,revenues_date_first_transaction,revenues_date_first_transaction_android,revenues_1d,...,revenues_2w,revenues_4w,revenues_8w,revenues_12w,revenues_16w,funnel_last_update,funnel_current,funnel_5min,funnel_1hour,funnel_1d,user_category,register_ip_region,register_ip_city,revenues_date_first_transaction_woe,revenues_incremental_fields_last_update,register_os,register_device,register_version,register_mobile_uid,register_mobile_device_aid,client_mobile_device_aid,last_global_device_id,register_device_android,register_source,register_source_android
10992,3.073215e+18,54280101.0,2015-05-08,2015-05-08,2015-05-10,android,2015-05-10,android,126.203.246.140,JP,35.6895,139.692,+09:00,4.0,92.0,5725.0,1689.0,1750.0,13.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2015-05-08,5004.0,5002.0,5004.0,5004.0,player,Tokyo,Tokyo,NaN,NaN,4.1.1,Sony Corporation WALKMAN,1505072029,14310770222599,NaN,28a451fc-f372-4d10-9465-13814577334c,28a451fc-f372-4d10-9465-13814577334c,Sony Corporation WALKMAN,game gift,game gift
11657,3.073215e+18,50664998.0,2015-05-08,2015-05-08,2015-05-08,android,2015-05-08,android,125.203.44.43,JP,35.6895,139.692,+09:00,6.0,33.0,11310.0,3926.0,6060.0,15.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2015-05-08,5006.0,5002.0,5004.0,5006.0,player,Tokyo,Tokyo,NaN,NaN,4.2.2,SHARP SHL22,1505072027,14310770219747,NaN,5766f9ee-7c53-4003-b0b0-2a8b2db211e2,5766f9ee-7c53-4003-b0b0-2a8b2db211e2,SHARP SHL22,game gift,game gift


In [28]:
import collections
collections.Counter(df_user['user_id'])

True

In [29]:
def valCount(lst):
    res = {}
    for v in lst:
        try:
            res[v] += 1
        except KeyError:
            res[v] = 1
    return res
lst = df_user['user_id']
u = [ x for x,y in valCount(lst).iteritems() if y > 1 ]

In [30]:
u

[3.0732143147949752e+18,
 3.0732217536783319e+18,
 3.0677491321195725e+18,
 3.0732145037735363e+18,
 3.0732381432735334e+18,
 3.0739893158737347e+18,
 3.0732153369971917e+18,
 3.0732125968080568e+18,
 3.0680862784623739e+18,
 3.0732393716341801e+18,
 3.0682328914659896e+18,
 3.0682372122030899e+18,
 3.0676754562505769e+18,
 3.0738129816964301e+18,
 3.0676793303110779e+18,
 3.0651502645886321e+18,
 3.0732152682777144e+18,
 3.0738057232016998e+18,
 3.0732136791398154e+18,
 3.0738371451824374e+18,
 3.0679838950319718e+18,
 3.0732234029457736e+18,
 3.0732259283865441e+18,
 3.0687666614116024e+18,
 3.0732131981034783e+18,
 3.0732140227371991e+18,
 3.0732194515758602e+18,
 3.0732152854575841e+18,
 3.0732184207837107e+18,
 3.067993610248e+18,
 3.0732209032748073e+18,
 3.0732213843111444e+18,
 3.0732211695627802e+18,
 3.0732245196372705e+18,
 3.0732276206036582e+18,
 3.0732300515551483e+18,
 3.0688620784050504e+18,
 3.073232310707945e+18,
 3.0732348533285847e+18,
 3.0738221385667046e+18,
 3.07

In [52]:
df_ses

,user_id,session_id,datetime,platform,version,sample_ratio,ip_ip,ip_country,ip_region,ip_city,ip_lat,ip_lon,ip_timezone,client_mobile_os,client_mobile_device,client_mobile_language,client_mobile_uid,client_mobile_device_aid,game_basic_level,game_basic_cash,game_resources_gold,game_resources_food,game_resources_xp,game_resources_stamina,game_resources_de,game_resources_ev,game_resources_fe,game_resources_friend_points,game_resources_le,game_resources_ne,game_resources_rde,game_resources_re,game_resources_rfe,game_resources_rle,game_resources_rne,game_resources_rwe,game_resources_ssre,game_resources_sre,game_resources_we
0,3.043009e+18,3.043009e+18,2014-11-26,android,0,1,62.97.102.90,ES,CATALONIA,BARCELONA,41.3888,2.15899,+02:00,NaN,NaN,NaN,NaN,NaN,1,5,3000.0,140,0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3.043009e+18,3.043009e+18,2014-11-26,android,0,1,62.97.102.90,ES,CATALONIA,BARCELONA,41.3888,2.15899,+02:00,NaN,NaN,NaN,NaN,NaN,2,10005,1002750.0,1000010,275,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3.043009e+18,3.043009e+18,2014-11-26,android,0,1,62.97.102.90,ES,CATALONIA,BARCELONA,41.3888,2.15899,+02:00,NaN,NaN,NaN,NaN,NaN,3,9926,998400.0,1000070,1075,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3.043009e+18,3.043009e+18,2014-11-26,android,0,1,62.97.102.90,ES,CATALONIA,BARCELONA,41.3888,2.15899,+02:00,NaN,NaN,NaN,NaN,NaN,7,9371,605526.0,1000120,8675,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3.043009e+18,3.043009e+18,2014-11-26,android,0,1,62.97.102.90,ES,CATALONIA,BARCELONA,41.3888,2.15899,+02:00,NaN,NaN,NaN,NaN,NaN,7,9371,605526.0,1000120,8675,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,3.043006e+18,3.043006e+18,2014-11-26,android,0,1,84.124.127.14,ES,MADRID,MADRID,40.4165,-3.70256,+02:00,NaN,NaN,NaN,NaN,NaN,1,5,3000.0,140,0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,3.043021e+18,3.043021e+18,2014-11-26,android,0,1,84.124.127.14,ES,MADRID,MADRID,40.4165,-3.70256,+02:00,NaN,NaN,NaN,NaN,NaN,1,5,3000.0,140,0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,3.043033e+18,3.043033e+18,2014-11-26,android,0,1,104.135.7.24,-,-,-,0.0000,0.00000,-,NaN,NaN,NaN,NaN,NaN,1,5,3000.0,140,0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,3.043161e+18,3.043161e+18,2014-11-27,android,0,1,62.97.102.90,ES,CATALONIA,BARCELONA,41.3888,2.15899,+02:00,NaN,NaN,NaN,NaN,NaN,1,5,3000.0,140,0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,3.043168e+18,3.043168e+18,2014-11-27,android,0,1,62.97.102.90,ES,CATALONIA,BARCELONA,41.3888,2.15899,+02:00,NaN,NaN,NaN,NaN,NaN,1,5,3000.0,140,0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [59]:
df_ses['datetime']

0        2014-11-26
1        2014-11-26
2        2014-11-26
3        2014-11-26
4        2014-11-26
5        2014-11-26
6        2014-11-26
7        2014-11-26
8        2014-11-27
9        2014-11-27
10       2014-11-27
11       2014-11-27
12       2014-11-27
13       2014-11-27
14       2014-11-27
15       2014-11-27
16       2014-11-28
17       2014-11-28
18       2014-11-28
19       2014-11-28
20       2014-11-28
21       2014-11-28
22       2014-11-28
23       2014-12-01
24       2014-12-02
25       2014-12-02
26       2014-12-02
27       2014-12-02
28       2014-12-03
29       2014-12-04
            ...    
148694   2015-06-22
148695   2015-06-22
148696   2015-06-22
148697   2015-06-22
148698   2015-06-22
148699   2015-06-22
148700   2015-06-22
148701   2015-06-22
148702   2015-06-22
148703   2015-06-22
148704   2015-06-22
148705   2015-06-22
148706   2015-06-22
148707   2015-06-22
148708   2015-06-22
148709   2015-06-22
148710   2015-06-22
148711   2015-06-22
148712   2015-06-22


In [65]:
df_user[['user_id','revenues_2w','funnel_1d']]

,user_id,revenues_2w,funnel_1d
0,3.042986e+18,NaN,NaN
1,1.234560e+05,NaN,NaN
2,3.043158e+18,NaN,201.0
3,3.043160e+18,NaN,NaN
4,3.043162e+18,NaN,201.0
5,3.043174e+18,NaN,201.0
6,3.043179e+18,NaN,NaN
7,3.043191e+18,NaN,NaN
8,3.043191e+18,NaN,NaN
9,3.043200e+18,NaN,NaN


In [75]:
df_ses[df_ses['user_id']==1.0]

,user_id,session_id,datetime,platform,version,sample_ratio,ip_ip,ip_country,ip_region,ip_city,ip_lat,ip_lon,ip_timezone,client_mobile_os,client_mobile_device,client_mobile_language,client_mobile_uid,client_mobile_device_aid,game_basic_level,game_basic_cash,game_resources_gold,game_resources_food,game_resources_xp,game_resources_stamina,game_resources_de,game_resources_ev,game_resources_fe,game_resources_friend_points,game_resources_le,game_resources_ne,game_resources_rde,game_resources_re,game_resources_rfe,game_resources_rle,game_resources_rne,game_resources_rwe,game_resources_ssre,game_resources_sre,game_resources_we
856,1.0,26852026.0,2015-03-30,ios,999999999999,1,62.97.102.90,ES,Catalonia,Barcelona,41.3888,2.15899,+01:00,None,None,None,None,None,1,32,6050.0,420,270,4,0.0,10.0,0.0,9.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
858,1.0,26852026.0,2015-03-30,ios,999999999999,1,62.97.102.90,ES,Catalonia,Barcelona,41.3888,2.15899,+01:00,None,None,None,None,None,1,32,6050.0,420,270,4,0.0,10.0,0.0,9.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
862,1.0,26852026.0,2015-03-30,ios,999999999999,1,62.97.102.90,ES,Catalonia,Barcelona,41.3888,2.15899,+01:00,None,None,None,None,None,1,32,6050.0,420,270,8,0.0,10.0,0.0,9.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
864,1.0,26852026.0,2015-03-30,ios,999999999999,1,62.97.102.90,ES,Catalonia,Barcelona,41.3888,2.15899,+01:00,None,None,None,None,None,1,32,6050.0,420,270,10,0.0,10.0,0.0,9.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
865,1.0,26852026.0,2015-03-30,ios,999999999999,1,62.97.102.90,ES,Catalonia,Barcelona,41.3888,2.15899,+01:00,None,None,None,None,None,1,32,6050.0,420,270,10,0.0,10.0,0.0,9.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
866,1.0,26852026.0,2015-03-30,ios,999999999999,1,62.97.102.90,ES,Catalonia,Barcelona,41.3888,2.15899,+01:00,None,None,None,None,None,1,32,6050.0,420,270,10,0.0,10.0,0.0,9.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
867,1.0,26852026.0,2015-03-30,ios,999999999999,1,62.97.102.90,ES,Catalonia,Barcelona,41.3888,2.15899,+01:00,None,None,None,None,None,1,32,6050.0,420,270,10,0.0,10.0,0.0,9.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
868,1.0,26852026.0,2015-03-30,ios,999999999999,1,62.97.102.90,ES,Catalonia,Barcelona,41.3888,2.15899,+01:00,None,None,None,None,None,1,32,6050.0,420,270,10,0.0,10.0,0.0,9.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
870,1.0,26852026.0,2015-03-30,ios,999999999999,1,62.97.102.90,ES,Catalonia,Barcelona,41.3888,2.15899,+01:00,None,None,None,None,None,1,32,6050.0,420,270,10,0.0,10.0,0.0,9.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
871,1.0,26852026.0,2015-03-30,ios,999999999999,1,62.97.102.90,ES,Catalonia,Barcelona,41.3888,2.15899,+01:00,None,None,None,None,None,1,32,6050.0,420,270,10,0.0,10.0,0.0,9.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
def cohort_period(df_ses):
    """
    Creates a `CohortPeriod` column, which is the Nth period based on the user's first purchase.
    
    Example
    -------
    Say you want to get the 3rd day for every user:
        df_ses.sort(['user_id', 'OrderTime', inplace=True)
        df_ses = df_ses.groupby('user_id').apply(cohort_period)
        df_ses[df_ses.CohortPeriod == 3]
    """
    df_ses['Cohort_period'] = np.arange(len(df_ses['datetime'])) + 1
    return df_ses

In [8]:
try1 = df_ses[['user_id','session_id','datetime']].groupby(df_ses['user_id']).apply(cohort_period)

In [30]:
try2 = try1[['user_id','session_id','datetime','Cohort_period']].groupby(df_ses['datetime']).agg()

TypeError: aggregate() takes at least 2 arguments (1 given)

In [28]:
try2

,user_id,session_id,datetime,Cohort_period
datetime,,,,
2014-11-26,34,34,34,34
2014-11-27,44,44,44,44
2014-11-28,17,17,17,17
2014-11-29,3,3,3,3
2014-12-01,2,2,2,2
2014-12-02,9,9,9,9
2014-12-03,8,8,8,8
2014-12-04,17,17,17,17
2014-12-05,25,25,25,25


In [21]:
to_drop = [1.0, 2.0]
df_ses = df_ses[~df_ses['user_id'].isin(to_drop)]

In [23]:
df_user[df_user['user_id']==1.0]

,user_id,last_session_id,date_register,date_register_android,date_last_logged,platform_last_logged,date_last_logged_android,register_platform,register_ip_ip,register_ip_country,register_ip_lat,register_ip_lon,register_ip_timezone,game_current_level,game_current_cash,game_current_gold,game_current_food,game_current_xp,game_current_stamina,revenues_dollars_net,revenues_dollars_gross,revenues_num_transactions,revenues_date_first_transaction,revenues_date_first_transaction_android,revenues_1d,...,revenues_2w,revenues_4w,revenues_8w,revenues_12w,revenues_16w,funnel_last_update,funnel_current,funnel_5min,funnel_1hour,funnel_1d,user_category,register_ip_region,register_ip_city,revenues_date_first_transaction_woe,revenues_incremental_fields_last_update,register_os,register_device,register_version,register_mobile_uid,register_mobile_device_aid,client_mobile_device_aid,last_global_device_id,register_device_android,register_source,register_source_android
1018,1.0,40933092.0,2015-03-30,2015-04-02,2015-04-08,ios,2015-04-02,ios,62.97.102.90,ES,41.3888,2.15899,+01:00,1.0,32.0,6050.0,420.0,270.0,10.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,player,Catalonia,Barcelona,NaN,NaN,None,None,999999999999,None,NaN,None,NaN,NaN,NaN,NaN


In [24]:
df_ses[df_ses['user_id']==2.0]

,user_id,session_id,datetime,platform,version,sample_ratio,ip_ip,ip_country,ip_region,ip_city,ip_lat,ip_lon,ip_timezone,client_mobile_os,client_mobile_device,client_mobile_language,client_mobile_uid,client_mobile_device_aid,game_basic_level,game_basic_cash,game_resources_gold,game_resources_food,game_resources_xp,game_resources_stamina,game_resources_de,game_resources_ev,game_resources_fe,game_resources_friend_points,game_resources_le,game_resources_ne,game_resources_rde,game_resources_re,game_resources_rfe,game_resources_rle,game_resources_rne,game_resources_rwe,game_resources_ssre,game_resources_sre,game_resources_we


In [108]:
df_ses['user_id'][5]

3.043005561127127e+18